In [4]:
import undetected_chromedriver as uc
import os
import time
import re
import chess

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Navigate to Computer Play

In [5]:
# Function to navigate to the "Play against computer" page
def navigate_to_computer_play(browser, delay=10):
    try:
        # Click the "Play with the computer" button by its visible text
        computer_button_xpath = "//button[contains(text(), 'Play with the computer')]"
        WebDriverWait(browser, delay).until(
            EC.presence_of_element_located((By.XPATH, computer_button_xpath))
        )
        computer_button = browser.find_element(By.XPATH, computer_button_xpath)
        ActionChains(browser).move_to_element(computer_button).click().perform()

        print("Navigating to the 'Play with the computer' page.")
    except TimeoutException:
        print("Navigating to 'Play with the computer' page: took too much time!")
    except Exception as e:
        print(f"Error navigating to 'Play with the computer': {e}")

In [6]:
# Function to select the computer level using JavaScript click
def select_computer_level(browser, level=1, delay=10):
    try:
        # XPath for the desired level based on its value (e.g., level 1, 2, 3)
        level_xpath = f"//input[@id='sf_level_{level}']"

        # Wait until the level radio button is present
        WebDriverWait(browser, delay).until(
            EC.presence_of_element_located((By.XPATH, level_xpath))
        )

        # Use JavaScript to click on the element
        level_radio = browser.find_element(By.XPATH, level_xpath)
        browser.execute_script(
            "arguments[0].scrollIntoView(true);", level_radio
        )  # Scroll into view
        browser.execute_script(
            "arguments[0].click();", level_radio
        )  # Perform the click using JS

        print(f"Computer level {level} selected.")
    except TimeoutException:
        print(f"Selecting computer level {level} took too much time!")
    except Exception as e:
        print(f"Error selecting computer level {level}: {e}")

In [7]:
def select_color(browser, color="random", delay=10):
    try:
        # Map color choices to their respective class names
        color_classes = {
            "random": "color-submits__button random",  # assuming random class
            "white": "color-submits__button white",
            "black": "color-submits__button black",
        }

        if color not in color_classes:
            raise ValueError(
                "Invalid color choice. Choose 'random', 'white', or 'black'."
            )

        # Get the class for the desired color
        color_class = color_classes[color]

        # XPath to locate the button with the specific class
        color_button_xpath = f"//button[contains(@class, '{color_class}')]"

        # Wait until the color button is present
        WebDriverWait(browser, delay).until(
            EC.presence_of_element_located((By.XPATH, color_button_xpath))
        )

        # Click the color button
        color_button = browser.find_element(By.XPATH, color_button_xpath)
        ActionChains(browser).move_to_element(color_button).click().perform()

        print(f"Color '{color}' selected.")
    except TimeoutException:
        print(f"Selecting color '{color}' took too much time!")
    except Exception as e:
        print(f"Error selecting color '{color}': {e}")

In [8]:
def generate_square_pixel_mapping(playing_as="white", timeout=5):
    """
    Generates a mapping of chess squares (e.g., 'a1', 'h8') to pixel coordinates
    (top-left and bottom-right corners) based on the board size.
    """
    try:
        # Wait until the chessboard element is present and visible
        chessboard = WebDriverWait(browser, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "cg-board"))
        )

        # Get the size of the chessboard (bounding box)
        board_size = chessboard.size
        board_width = board_size["width"]
        board_height = board_size["height"]

        square_width = board_width / 8
        square_height = board_height / 8

        # Files and ranks in chess
        files = "abcdefgh"
        ranks = "12345678"  # Rank 1 at the bottom (for white at the bottom)

        if playing_as == "black":
            files = files[::-1]
            ranks = ranks[::-1]

        # Dictionary to store the pixel coordinates for each square
        square_pixel_mapping = {}

        # Generate the pixel coordinates for each square
        for rank_index, rank in enumerate(ranks):
            for file_index, file in enumerate(files):
                # Calculate the pixel coordinates for the top-left corner of the square
                top_left_x = file_index * square_width
                top_left_y = (
                    7 - rank_index
                ) * square_height  # Flip y-coordinate for ranks

                # Calculate the pixel coordinates for the bottom-right corner of the square
                bottom_right_x = top_left_x + square_width
                bottom_right_y = top_left_y + square_height

                # Store the pixel coordinates for this square
                square_pixel_mapping[f"{file}{rank}"] = {
                    "top_left": (top_left_x, top_left_y),
                    "bottom_right": (bottom_right_x, bottom_right_y),
                }

        return square_pixel_mapping

    except TimeoutException:
        print(f"Chessboard element not found within {timeout} seconds.")
        return None

In [9]:
sqpm = generate_square_pixel_mapping(playing_as="white")
sqpm

{'a1': {'top_left': (0.0, 469.0), 'bottom_right': (67.0, 536.0)},
 'b1': {'top_left': (67.0, 469.0), 'bottom_right': (134.0, 536.0)},
 'c1': {'top_left': (134.0, 469.0), 'bottom_right': (201.0, 536.0)},
 'd1': {'top_left': (201.0, 469.0), 'bottom_right': (268.0, 536.0)},
 'e1': {'top_left': (268.0, 469.0), 'bottom_right': (335.0, 536.0)},
 'f1': {'top_left': (335.0, 469.0), 'bottom_right': (402.0, 536.0)},
 'g1': {'top_left': (402.0, 469.0), 'bottom_right': (469.0, 536.0)},
 'h1': {'top_left': (469.0, 469.0), 'bottom_right': (536.0, 536.0)},
 'a2': {'top_left': (0.0, 402.0), 'bottom_right': (67.0, 469.0)},
 'b2': {'top_left': (67.0, 402.0), 'bottom_right': (134.0, 469.0)},
 'c2': {'top_left': (134.0, 402.0), 'bottom_right': (201.0, 469.0)},
 'd2': {'top_left': (201.0, 402.0), 'bottom_right': (268.0, 469.0)},
 'e2': {'top_left': (268.0, 402.0), 'bottom_right': (335.0, 469.0)},
 'f2': {'top_left': (335.0, 402.0), 'bottom_right': (402.0, 469.0)},
 'g2': {'top_left': (402.0, 402.0), 'botto

In [10]:
def find_square_for_pixel(x, y, square_pixel_mapping):
    """
    Finds the corresponding chess square for pixel coordinates (x, y).
    """
    for square, boundaries in square_pixel_mapping.items():
        top_left = boundaries["top_left"]
        bottom_right = boundaries["bottom_right"]

        # Extract pixel coordinates for the top-left and bottom-right corners
        top_left_x, top_left_y = top_left
        bottom_right_x, bottom_right_y = bottom_right

        # Check if the x, y coordinates fall within this square's pixel range
        if top_left_x <= x < bottom_right_x and top_left_y <= y < bottom_right_y:
            return square

    return None  # If no square found (this shouldn't happen on a valid board)

In [11]:
find_square_for_pixel(248, 186, sqpm)

'd6'

In [12]:
from selenium.common.exceptions import NoSuchElementException, TimeoutException


def extract_piece_positions(timeout=10):
    """
    Extracts the piece positions from the chessboard (<cg-board>) by reading the
    transform: translate(x, y) values and the piece class.

    Parameters:
    - timeout: Maximum time to wait for the piece elements to be present.

    Returns:
    - piece_positions: A list of dictionaries with piece class and its (x, y) coordinates.
    """
    piece_positions = []

    try:
        # Wait for the piece elements on the chessboard
        pieces = WebDriverWait(browser, timeout).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "cg-board piece"))
        )

        # Loop through each piece found on the board
        for piece in pieces:
            # Get the class name of the piece, e.g., 'white queen', 'black king'
            piece_class = piece.get_attribute("class")

            # Extract the transform: translate(x, y) values using a regular expression
            transform_style = piece.get_attribute("style")
            match = re.search(r"translate\((\d+)px, (\d+)px\)", transform_style)

            if match:
                x = int(match.group(1))  # x-coordinate
                y = int(match.group(2))  # y-coordinate

                # Add the piece info to the list
                piece_positions.append({"piece": piece_class, "x": x, "y": y})

    except (NoSuchElementException, TimeoutException):
        print(f"Piece elements not found within {timeout} seconds.")

    except Exception as e:
        print(f"An error occurred: {e}")

    return piece_positions


def map_pieces_to_squares(playing_as="white"):
    """
    Maps pieces (based on their pixel positions) to chess squares.

    Parameters:
    - playing_as: Whether the player is playing as "white" or "black"
    """
    # Generate the mapping of all chess squares to pixel coordinates
    square_pixel_mapping = generate_square_pixel_mapping(playing_as)

    piece_square_mapping = {}

    # Find the piece positions from the chessboard (cg-board)
    piece_positions = extract_piece_positions()

    # Loop through each piece and find the corresponding square
    for piece in piece_positions:
        x = piece["x"]
        y = piece["y"]

        # Find the square for the piece's (x, y) coordinates
        square = find_square_for_pixel(x, y, square_pixel_mapping)

        if square:
            piece_square_mapping[square] = piece["piece"]

    return piece_square_mapping

In [13]:
map_pieces_to_squares(playing_as="white")

{'a8': 'black rook',
 'b8': 'black knight',
 'c8': 'black bishop',
 'd8': 'black queen',
 'e8': 'black king',
 'f8': 'black bishop',
 'g8': 'black knight',
 'h8': 'black rook',
 'a7': 'black pawn',
 'b7': 'black pawn',
 'c7': 'black pawn',
 'd7': 'black pawn',
 'e7': 'black pawn',
 'f7': 'black pawn',
 'g7': 'black pawn',
 'h7': 'black pawn',
 'a2': 'white pawn',
 'b2': 'white pawn',
 'c2': 'white pawn',
 'd2': 'white pawn',
 'e2': 'white pawn',
 'f2': 'white pawn',
 'g2': 'white pawn',
 'h2': 'white pawn',
 'a1': 'white rook',
 'b1': 'white knight',
 'c1': 'white bishop',
 'd1': 'white queen',
 'e1': 'white king',
 'f1': 'white bishop',
 'g1': 'white knight',
 'h1': 'white rook'}

In [14]:
pixel_mapping = generate_square_pixel_mapping(playing_as="black")
print(pixel_mapping)

{'h8': {'top_left': (0.0, 469.0), 'bottom_right': (67.0, 536.0)}, 'g8': {'top_left': (67.0, 469.0), 'bottom_right': (134.0, 536.0)}, 'f8': {'top_left': (134.0, 469.0), 'bottom_right': (201.0, 536.0)}, 'e8': {'top_left': (201.0, 469.0), 'bottom_right': (268.0, 536.0)}, 'd8': {'top_left': (268.0, 469.0), 'bottom_right': (335.0, 536.0)}, 'c8': {'top_left': (335.0, 469.0), 'bottom_right': (402.0, 536.0)}, 'b8': {'top_left': (402.0, 469.0), 'bottom_right': (469.0, 536.0)}, 'a8': {'top_left': (469.0, 469.0), 'bottom_right': (536.0, 536.0)}, 'h7': {'top_left': (0.0, 402.0), 'bottom_right': (67.0, 469.0)}, 'g7': {'top_left': (67.0, 402.0), 'bottom_right': (134.0, 469.0)}, 'f7': {'top_left': (134.0, 402.0), 'bottom_right': (201.0, 469.0)}, 'e7': {'top_left': (201.0, 402.0), 'bottom_right': (268.0, 469.0)}, 'd7': {'top_left': (268.0, 402.0), 'bottom_right': (335.0, 469.0)}, 'c7': {'top_left': (335.0, 402.0), 'bottom_right': (402.0, 469.0)}, 'b7': {'top_left': (402.0, 402.0), 'bottom_right': (469

In [15]:
map_pieces_to_squares(playing_as="white")["e1"]

'white king'

In [16]:
from selenium.common.exceptions import NoSuchElementException, TimeoutException


def find_piece_element_for_square(square, square_pixel_mapping, timeout=10):
    """
    Finds the piece element on the chessboard corresponding to a given square based on the pixel mapping.

    Parameters:
    - square: The chess square (e.g., 'd4') to look for a piece.
    - square_pixel_mapping: A dictionary mapping squares to their pixel boundaries.
    - timeout: Maximum time to wait for the piece elements to be present.

    Returns:
    - The piece WebElement found on the specified square, or None if no piece is found.
    """
    try:
        # Get the pixel boundaries for the specified square
        boundaries = square_pixel_mapping.get(square, {})
        print(boundaries)
        if not boundaries:
            return None

        top_left_x, top_left_y = boundaries["top_left"]
        bottom_right_x, bottom_right_y = boundaries["bottom_right"]

        # Wait for the pieces to be present on the board
        pieces = WebDriverWait(browser, timeout).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "cg-board piece"))
        )

        # Loop through each piece to find the one matching the square's pixel boundaries
        for piece in pieces:
            transform_style = piece.get_attribute("style")
            match = re.search(r"translate\((\d+)px, (\d+)px\)", transform_style)

            if match:
                x = int(match.group(1))
                y = int(match.group(2))

                # Check if the piece is within the boundaries of the square
                if (
                    top_left_x <= x < bottom_right_x
                    and top_left_y <= y < bottom_right_y
                ):
                    return piece

    except (NoSuchElementException, TimeoutException):
        print(f"Piece elements not found for square {square} within {timeout} seconds.")

    except Exception as e:
        print(f"An error occurred while finding the piece on square {square}: {e}")

    return None


def drag_and_drop_by_square(start_square, end_square, playing_as="white"):
    try:
        # Generate the pixel mappings
        square_pixel_mapping = generate_square_pixel_mapping(playing_as)

        # Find the starting piece element
        start_piece = find_piece_element_for_square(start_square, square_pixel_mapping)
        if not start_piece:
            print(f"Error: No piece found at {start_square}.")
            return

        # Get pixel coordinates for the start and end squares
        start_pixel = square_pixel_mapping.get(start_square, {}).get("top_left")
        end_pixel = square_pixel_mapping.get(end_square, {}).get("top_left")

        if not start_pixel or not end_pixel:
            print("Error: Invalid square notation.")
            return

        # Create an ActionChains object
        actions = ActionChains(browser)

        # Perform the drag-and-drop action using the piece element
        actions.click_and_hold(start_piece).move_by_offset(
            end_pixel[0] - start_pixel[0], end_pixel[1] - start_pixel[1]
        ).release().perform()

        print(f"Dragged from {start_square} to {end_square}.")

    except Exception as e:
        print(f"Error in drag and drop by square: {e}")

In [17]:
# # Example usage: Drag from f6 to f5
# drag_and_drop_by_square("e2", "e4", playing_as="white")

In [28]:
move_san = "g8=Q"
promotion_match = re.match(r"([a-h][18])=(Q|N|R|B)([+#]?)", move_san)

if not promotion_match:
    print("No prmotion detected")
    

In [18]:
import re
from selenium.common.exceptions import NoSuchElementException


def handle_pawn_promotion(browser, move_san):
    """
    Detects if the move is a pawn promotion (e.g., g8=Q), and selects the correct
    promotion piece in the UI by clicking the appropriate element.

    Parameters:
    - browser: The Selenium WebDriver instance.
    - move_san: The SAN string of the move, e.g., 'g8=Q' for a promotion to queen.
    """
    promotion_match = re.match(r"([a-h][18])=(Q|N|R|B)([+#]?)", move_san)

    if not promotion_match:
        print("No prmotion detected")
        return

    else:

        promotion_square, promotion_piece, check_or_mate = promotion_match.groups()
        print(
            f"Promotion to {promotion_piece} on {promotion_square} with {check_or_mate if check_or_mate else 'no check or checkmate'}"
        )

        # Wait until the promotion choice UI is visible (with the correct ID)
        try:
            promotion_choice_element = WebDriverWait(browser, 10).until(
                EC.visibility_of_element_located((By.ID, "promotion-choice"))
            )
        except TimeoutException:
            print("Promotion choice element not found.")
            return

        # Define a mapping of piece notation to the expected CSS class
        piece_class_mapping = {"Q": "queen", "N": "knight", "R": "rook", "B": "bishop"}

        # Get the piece class to select (e.g., 'queen', 'knight', etc.)
        piece_class = piece_class_mapping.get(promotion_piece)

        if not piece_class:
            print(f"Invalid promotion piece: {promotion_piece}")
            return

        # Find the correct piece element to click within the promotion choice UI
        try:
            # The pieces are inside squares, so find the correct square
            promotion_squares = promotion_choice_element.find_elements(
                By.CSS_SELECTOR, "square"
            )

            for square in promotion_squares:
                piece = square.find_element(By.CSS_SELECTOR, "piece")
                piece_class_attribute = piece.get_attribute("class")

                if piece_class in piece_class_attribute:
                    # Found the correct promotion piece, now click on it
                    square.click()
                    print(f"Clicked on promotion piece: {piece_class}")
                    break
            else:
                print(f"Could not find the promotion piece: {promotion_piece}")
        except NoSuchElementException:
            print(f"Error: Could not find elements for promotion selection.")

In [19]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException


def detect_last_table_move_and_count(playing_as="white", max_wait=10):
    """
    Detects the computer's move from the move table and ensures the move count has increased.

    Parameters:
    - board: The current board object to parse the move into UCI format.
    - previous_move_count: The total number of moves detected previously.
    - max_wait: Maximum time (in seconds) to wait for the move table to appear.

    Returns:
    - computer_move: The computer's latest move in SAN format.
    - current_move_count: The updated move count.
    """
    try:
        # Wait until the move history table is present and visible
        move_table = WebDriverWait(browser, max_wait).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "l4x"))
        )

        # Find all move elements in the table (each move is in a <kwdb> tag)
        move_elements = move_table.find_elements(By.CSS_SELECTOR, "kwdb")

        # Get the total count of moves from all elements (both white and black)
        current_move_count = len(move_elements)

        # Get the last move element (latest move made)
        if playing_as == "white":

            last_move_element = move_elements[-1]
        else:
            last_move_element = move_elements[-2]

        computer_move = last_move_element.text.strip()  # Extract the move text

        # Return the detected move and the updated move count
        return computer_move, current_move_count

    except (NoSuchElementException, TimeoutException):
        print("Move history table or move element not found.")
        return None, None

In [20]:
def wait_for_computer_move(
    previous_move, previous_move_count, playing_as="white", max_wait=10
):
    """
    Loops until a new computer move is detected, based on move count and the difference
    from the previous move.

    Parameters:
    - board: The current board object to parse the move into UCI format.
    - previous_move: The last detected move (to avoid returning the same move).
    - previous_move_count: The total number of moves detected previously.
    - max_wait: Maximum time (in seconds) to wait for each iteration of move detection.

    Returns:
    - computer_move: The new computer move.
    - current_move_count: The updated move count.
    """
    while True:
        # Detect the current move and move count
        last_move, current_move_count = detect_last_table_move_and_count(
            playing_as, max_wait
        )

        # Check if the move count has increased and the last move is different
        if current_move_count > previous_move_count and last_move != previous_move:
            print(f"New computer move detected: {last_move}")
            return last_move

        # Optional: Print status for debugging
        print(
            f"Waiting for new computer move... (Current move count: {current_move_count})"
        )

        # Small delay before the next check to avoid excessive CPU usage
        time.sleep(0.1)

In [21]:
def translate_move_to_web(move):
    """
    Translate a chess move to web-based interaction.

    Parameters:
    - move: The move to translate in UCI format (e.g., 'e2e4').

    Returns:
    - The start and end squares as tuples (e.g., ('e2', 'e4')).
    """
    start_square = move[:2]
    end_square = move[2:4]
    return start_square, end_square

In [25]:
def play_game(engine, playing_as="white", timeout=100):
    """
    Play a game against the computer by interacting with the web-based chessboard.

    Parameters:
    - engine: The chess engine object.
    - playing_as: Whether the player is "white" or "black".
    - timeout: Maximum time (in seconds) to wait for the computer's move.
    """
    board = chess.Board()
    previous_move = None
    game_move_count = 0  # Track the number of moves in the move history

    # Handle initial computer move when playing as Black
    if playing_as == "black":
        time.sleep(1)  # Give time for the initial move to appear
        last_table_move, table_move_count = detect_last_table_move_and_count(
            playing_as=playing_as, max_wait=timeout
        )
        print("move count according to table", table_move_count)
        print("move count according to game logic", game_move_count)

        computer_move = wait_for_computer_move(
            previous_move,
            game_move_count,
            max_wait=10,
            playing_as=playing_as,
        )
        if computer_move:
            print(f"Detected initial computer move: {last_table_move}")
            board.push_san(last_table_move)  # Push the move to the board object
            engine.set_fen_position(board.fen())  # Update engine's FEN position
            print("Evaluation after initial computer move:", engine.get_evaluation())

    # Main game loop
    while not board.is_game_over():
        # Get the best move from the engine for the current board position
        engine.set_fen_position(board.fen())
        best_move = engine.get_top_moves(3)[0]["Move"]
        print(f"Best move: {best_move}")

        # Translate the best move to a web-based interaction (start and end squares)
        start_square, end_square = translate_move_to_web(best_move)
        print(f"Performing move: {start_square} to {end_square}")
        drag_and_drop_by_square(start_square, end_square)
        handle_pawn_promotion(browser, best_move)
        last_engine_move = best_move
        game_move_count += 1
        # Push the best move to the board
        board.push_san(best_move)
        if board.is_checkmate():
            print("MY BOT CHECKMATED YOU HAA")
            break

        last_table_move, table_move_count = detect_last_table_move_and_count(
            playing_as=playing_as
        )
        print("move count according to table", table_move_count)
        print("move count according to game logic", game_move_count)

        computer_move = wait_for_computer_move(
            previous_move,
            game_move_count,
            max_wait=10,
            playing_as=playing_as,
        )

        if computer_move:
            # Computer's move detected, push it to the board and update engine
            print(f"Detected computer move: {computer_move}")
            board.push_san(computer_move)
            if board.is_checkmate():
                print("MY BOT GOT CHECKMATED :(")
                break
            engine.set_fen_position(board.fen())
            game_move_count += 1
            print("Evaluation after computer move:", engine.get_evaluation())
            # Update move count

        else:
            # Handle case where computer didn't make a move within the timeout
            print("Computer did not make a move in the given time.")
            break

        # Check if the game is over after the move
        if board.is_game_over():
            print("Game over.")
            break

    engine.send_quit_command()

In [26]:
from stockfish import Stockfish

# https://github.com/zhelyabuzhsky/stockfish/blob/master/stockfish/models.py
engine = Stockfish(
    "../stockfish-ubuntu-x86-64-bmi2/stockfish/stockfish-ubuntu-x86-64-bmi2"
)
engine.set_depth(20)  # How deep the AI looks
engine.set_skill_level(20)  # Highest rank stockfish
engine.get_parameters()

{'Debug Log File': '',
 'Contempt': 0,
 'Min Split Depth': 0,
 'Ponder': 'false',
 'MultiPV': 1,
 'Skill Level': 20,
 'Move Overhead': 10,
 'Minimum Thinking Time': 20,
 'Slow Mover': 100,
 'UCI_Chess960': 'false',
 'UCI_LimitStrength': 'false',
 'UCI_Elo': 1350,
 'Threads': 1,
 'Hash': 16}

In [ ]:
uddir = "/Users/mdicio/Library/Application Support/Google/Chrome"
profile = "Profile 1"

if os.path.exists(f"{uddir}/{profile}"):
    print(1)

u_options = uc.ChromeOptions()
u_options.add_argument(f"user-data-dir={uddir}")
u_options.add_argument(f"--profile-directory={profile}")

browser = uc.Chrome(
    browser_executable_path="/usr/bin/google-chrome-beta",
    options=u_options,
    use_subprocess=True,
)
time.sleep(2.8)
HOME_URL = "https://lichess.org/"
browser.get(HOME_URL)

In [ ]:
# navigate_to_computer_play(browser)
# select_computer_level(browser, level=8)
# select_color(browser, color="white")

In [27]:
play_game(engine, playing_as="white")

Best move: e2e4
Performing move: e2 to e4
{'top_left': (268.0, 402.0), 'bottom_right': (335.0, 469.0)}
Dragged from e2 to e4.
move count according to table 1
move count according to game logic 1
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting

KeyboardInterrupt: 